In [ ]:
"""
Random Forest
Global Feature Selection 
    - X = 'Depth (m)','iso_classes_encoded', 'Oberhollenzer_classes', 
    'σ,v (kPa)', 'σ',v (kPa)'.
    
    - Y = 'qc (MPa)','fs (kPa)'.

Link: https://github.com/DowellChan/ResNetRegression
"""
#General Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, CSVLogger
from sklearn.model_selection import train_test_split
import time

'''
PreProcessing
''' 
#Loading Database
df_cpt=pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/Target_Encoded_Database.csv')

#Transforming to DataFrame
df_cpt = pd.DataFrame(df_cpt)
df_cpt = df_cpt.drop(columns='Unnamed: 0')

#Dropping zeros from qc
df_cpt = df_cpt[~(df_cpt[['qc (MPa)']] == 0).any(axis = 1)]
df_cpt = df_cpt[~(df_cpt[['qc (MPa)']] < 0).any(axis = 1)]
df_cpt = df_cpt[~(df_cpt[['SBT (-)']] == 0).any(axis = 1)]
df_cpt = df_cpt[~(df_cpt[['SBTn (-)']] == 0).any(axis = 1)]
df_cpt = df_cpt[~(df_cpt[['Mod. SBTn (-)']] == 0).any(axis = 1)]

#Transforming to NumpyArray
df_cpt = df_cpt.to_numpy()

'''
Inputs
'''
X = df_cpt[:, [11, 17, 18, 19, 27]]

#1--> 'Depth (m)' --> index: 1
#2 --> 'σ,v (kPa)' --> index: 9
#3 --> "σ',v (kPa)" --> index: 11
#4 --> 'SBT (-)' --> index: 17
#5 --> 'SBTn (-)' --> index: 18
#6 --> 'Mod. SBTn (-)' --> index: 19 
#7 --> 'Ic (-)' --> index: 21
#8 --> 'Ic SBT (-)' --> index:22 
#9 --> 'Ib (-)' --> index: 23
#10 --> 'Oberhollenzer_classes' --> index: 24
#11 --> 'iso_classes_encoded' --> index: 27

#X=X.to(device) #To GPU

'''
Outputs
'''
y = df_cpt[:, [2, 3]] 

#1 --> 'qc (MPa)' --> index: 2
#2 --> 'fs (kPa)' --> index: 3

#y=y.to(device) #To GPU

#Scaling the and preparing the data
#scalerX = MinMaxScaler()
#scalerY = MinMaxScaler()

#Scaling the Data
#scaledX=scalerX.fit_transform(X)
#scaledY=scalerY.fit_transform(y)

#Train & Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size = 0.20)

In [ ]:
#CrossVal
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
#Random forest Model
model = RandomForestRegressor(bootstrap = 'True', n_estimators = 1400, max_depth = 300,
                              min_samples_split = 15, min_samples_leaf = 2)

#K-Fold
cv = 7

In [ ]:
from sklearn.model_selection import cross_validate
#Initial time
start_time = time.time()

#Evaluating the model
#scores = cross_val_score(model, X_train, y_train, cv = cv, scoring = ['neg_mean_squared_error', 'r2_score'])

scores = cross_validate(model, X_train, y_train, cv = cv, scoring = ('r2', 'neg_mean_squared_error'), return_train_score = True)

#Final time
end_time = (time.time() - start_time)
print('The running time is: %.2f minutes' %(end_time/60))

The running time is: 390.46 minutes


In [ ]:
print(scores['train_r2'])

[0.91065356 0.91071873 0.9104604  0.91088463 0.91159484 0.91098452
 0.91081305]


In [ ]:
print(scores)

{'fit_time': array([2834.56946206, 2866.90892768, 2949.6161685 , 2974.68323708,
       2889.85171556, 2930.7768805 , 2983.44622803]), 'score_time': array([59.78956008, 64.31604528, 65.65402365, 63.72284961, 60.81736827,
       61.46363831, 62.34236193]), 'test_r2': array([0.87049388, 0.87046187, 0.87137807, 0.86525126, 0.86309645,
       0.86702744, 0.8668328 ]), 'train_r2': array([0.91065356, 0.91071873, 0.9104604 , 0.91088463, 0.91159484,
       0.91098452, 0.91081305]), 'test_neg_mean_squared_error': array([-446.8935057 , -444.99808677, -447.35475369, -471.96881879,
       -474.18184124, -461.6679224 , -470.23820502]), 'train_neg_mean_squared_error': array([-302.41462186, -302.25970897, -302.52188703, -301.01610083,
       -299.21356406, -300.90159415, -300.67391578])}
